In [3]:
import os

from dotenv import load_dotenv

load_dotenv()

True

In [57]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage

llm = ChatOpenAI(model="gpt-3.5-turbo")

messages = [
    HumanMessage(content="List 8 wonders of the word")
]

result = llm(messages)

wonders_list = result.content.split("\n")

for wonder in wonders_list:
    print(wonder)

1. Great Wall of China
2. Petra, Jordan
3. Christ the Redeemer, Brazil
4. Machu Picchu, Peru
5. Taj Mahal, India
6. Colosseum, Italy
7. Chichen Itza, Mexico
8. Great Pyramid of Giza, Egypt


## Streaming and Batching


In [ ]:
for chunk in llm.stream("8 Wonders of World"):
    print(chunk.content)


1
.
 Great
 Wall
 of
 China


2
.
 Petra
,
 Jordan


3
.
 Christ
 the
 Rede
emer
,
 Brazil


4
.
 Mach
u
 Pic
chu
,
 Peru


5
.
 Ch
ichen
 It
za
,
 Mexico


6
.
 Roman
 Col
os
se
um
,
 Italy


7
.
 Taj
 Mah
al
,
 India


8
.
 Great
 Pyramid
 of
 G
iza
,
 Egypt



In [ ]:
result = llm.batch(
    [
        "What's 2 +2 ?",
        "What's 34.9 + 23.78?"
    ]
)
for _result in result:
    print(_result.content)

2 + 2 equals 4.
58.68


## System & Human Message

In [ ]:
from langchain.schema import SystemMessage

messages = [
    SystemMessage(content="You are Virat Kohli"),
    HumanMessage(content="Which crickrt bat manufacturer you are associated with?")
]

result = llm(messages)
result.content

'I am associated with the cricket bat manufacturer, MRF.'

## Prompt Template

In [ ]:
from langchain.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    "tell me a {objective} joke about {content}"
)

filled_prompt = prompt_template.format(objective="funny", content="robots")

response = llm.invoke(filled_prompt)
response.content

'Why was the robot so bad at soccer? Because it kept kicking up sparks instead of the ball!'

## Chat Prompt Template

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage

# Defining a chat template with various roles
chat_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content="You are a helpful AI Mathematics teacher bot. Your name is {name}"),
        HumanMessage(content="Hello, How are you doing?"),
        AIMessage(content="I'm doing well, thanks!"),
        HumanMessage(content="{math_problem}")
    ]
)

math_problem = input("Enter Math problem:")

formatted_messages = chat_template.format_messages(name="Bob", math_problem=math_problem)

for message in formatted_messages:
    print(message.content)

You are a helpful AI Mathematics teacher bot. Your name is {name}
Hello, How are you doing?
I'm doing well, thanks!
{math_problem}


## Output Parsers

### Simple Json Parser

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.output_parsers.json import SimpleJsonOutputParser

json_parser = SimpleJsonOutputParser()

template = """
Answer the user's question:
{question}
Return the answer as a JSON object with keys 'birth_place' and 'birth_date'.
"""

json_prompt = PromptTemplate.from_template(template)

json_chain = json_prompt | llm | json_parser

input_data = {"question": "When and where were Elon Musk and Bill Gates born?"}

output = json_chain.invoke(input_data)
output

{'birth_place': {'Elon Musk': 'Pretoria, South Africa',
  'Bill Gates': 'Seattle, Washington, United States'},
 'birth_date': {'Elon Musk': 'June 28, 1971',
  'Bill Gates': 'October 28, 1955'}}

### CSV Parser

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.output_parsers import CommaSeparatedListOutputParser

csv_parser = CommaSeparatedListOutputParser()

template = """
Answer the user's question:
{question}
Return the answer as a comma-separated list with the format: "Country, Capital, Independence Date".
Only include countries in Asia.
"""

csv_prompt = PromptTemplate.from_template(template)

csv_chain = csv_prompt | llm | csv_parser

input_data = {"question": "Give details of countries in Aisa"}

output = csv_chain.invoke(input_data)
output

['Afghanistan',
 'Kabul',
 'August 19',
 '1919\nBahrain',
 'Manama',
 'August 15',
 '1971\nBangladesh',
 'Dhaka',
 'December 16',
 '1971\nBhutan',
 'Thimphu',
 'August 8',
 '1949\nBrunei',
 'Bandar Seri Begawan',
 'January 1',
 '1984\nCambodia',
 'Phnom Penh',
 'November 9',
 '1953\nChina',
 'Beijing',
 'October 1',
 '1949\nIndia',
 'New Delhi',
 'August 15',
 '1947\nIndonesia',
 'Jakarta',
 'August 17',
 '1945\nIran',
 'Tehran',
 'February 1',
 '1979\nIraq',
 'Baghdad',
 'October 3',
 '1932\nIsrael',
 'Jerusalem',
 'May 14',
 '1948\nJapan',
 'Tokyo',
 'May 3',
 '1947\nJordan',
 'Amman',
 'May 25',
 '1946\nKazakhstan',
 'Nur-Sultan',
 'December 16',
 '1991\nKuwait',
 'Kuwait City',
 'June 19',
 '1961\nKyrgyzstan',
 'Bishkek',
 'August 31',
 '1991\nLaos',
 'Vientiane',
 'July 19',
 '1949\nLebanon',
 'Beirut',
 'November 22',
 '1943\nMalaysia',
 'Kuala Lumpur',
 'August 31',
 '1957\nMaldives',
 'Malé',
 'July 26',
 '1965\nMongolia',
 'Ulaanbaatar',
 'December 29',
 '1911\nMyanmar',
 'Nay

## Retrieval

In [ ]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("/home/arslanhaider/ahs-data/test-pdf.pdf")

pages = loader.load_and_split()

pages[0].page_content.split("\n")

['Welcome to Smallpdf',
 'Digital Documents—All In One Place',
 'Access Files Anytime, Anywhere Enhance Documents in One Click ',
 'Collaborate With Others With the new Smallpdf experience, you can ',
 'freely upload, organize, and share digital ',
 'documents. When you enable the ‘Storage’ ',
 'option, we’ll also store all processed files here. ',
 'You can access files stored on Smallpdf from ',
 'your computer, phone, or tablet. We’ll also ',
 'sync files from the Smallpdf Mobile App to our ',
 'online portalWhen you right-click on a file, we’ll present ',
 'you with an array of options to convert, ',
 'compress, or modify it. ',
 'Forget mundane administrative tasks. With ',
 'Smallpdf, you can request e-signatures, send ',
 'large files, or even enable the Smallpdf G Suite ',
 'App for your entire organization. Ready to take document management to the next level?']

## Agent

In [ ]:
import json
from datetime import datetime

import requests
from langchain.agents import initialize_agent, Tool

# Define tools as functions
def read_tasks_from_file(input):
    with open('tasks.json', 'r') as file:
        tasks = json.load(file)
    return tasks

def check_weather(date):
    response = requests.get(f"https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/London,UK/{datetime.now().strftime('%Y-%m-%d')}?key={os.getenv('WEATHER_API_KEY')}")
    return response.json()

def add_to_calendar(task):
    print(f"Adding task to calendar: {task}")
    return f"Task {task} added to calendar."

# Define the tools for the agent
tools = [
    Tool(
        name="read_tasks_from_file",
        func=read_tasks_from_file,
        description="Reads tasks from a local JSON file"
    ),
    Tool(
        name="check_weather",
        func=check_weather,
        description="Checks the weather forecast for a given date"
    ),
    Tool(
        name="add_to_calendar",
        func=add_to_calendar,
        description="Adds a task to the calendar"
    )
]

agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)


sample_tasks = [
    {"title": "Buy groceries", "tags": ["shopping"]},
    {"title": "Jogging", "tags": ["outdoor", "exercise"]},
    {"title": "Team meeting", "tags": ["work"]},
]
with open('tasks.json', 'w') as file:
    json.dump(sample_tasks, file)

input_text = "Manage my tasks for today. Check if I can go jogging, and schedule my team meeting."

# Plan and execute the tasks
result = agent(input_text)

# Display the result
print(result)




> Entering new AgentExecutor chain...
I need to check the weather to see if I can go jogging and then schedule the team meeting.
Action: check_weather
Action Input: today's date
Observation: {'queryCost': 1, 'latitude': 51.5064, 'longitude': -0.12721, 'resolvedAddress': 'London, England, United Kingdom', 'address': 'London,UK', 'timezone': 'Europe/London', 'tzoffset': 1.0, 'description': 'Similar temperatures continuing with no rain expected.', 'days': [{'datetime': '2024-07-18', 'datetimeEpoch': 1721257200, 'tempmax': 80.5, 'tempmin': 57.8, 'temp': 69.0, 'feelslikemax': 80.5, 'feelslikemin': 57.8, 'feelslike': 69.1, 'dew': 56.2, 'humidity': 65.9, 'precip': 0.0, 'precipprob': 0.0, 'precipcover': 0.0, 'preciptype': ['rain'], 'snow': 0.0, 'snowdepth': 0.0, 'windgust': 21.9, 'windspeed': 10.1, 'winddir': 167.0, 'pressure': 1020.3, 'cloudcover': 70.8, 'visibility': 8.4, 'solarradiation': 247.0, 'solarenergy': 21.4, 'uvindex': 7.0, 'severerisk': 10.0, 'sunrise': '05:05:13', 'sunriseEpoch'

### Use Built-in Tools for Agents

In [48]:
# calculator Tool
from langchain.agents import initialize_agent, Tool
from langchain.tools import DuckDuckGoSearchRun

# Initialize the DuckDuckGoSearchRun tool
duckduckgo_tool = DuckDuckGoSearchRun()

# Define a tool for performing a web search
def perform_web_search(query):
    try:
        search_results = duckduckgo_tool.run(query)
        return search_results
    except Exception as e:
        return f"Error: {str(e)}"
    
tools = [
    Tool(
        name="web_search",
        func=perform_web_search,
        description="Performs a web search using DuckDuckGo"
    ),
    Tool(
        name="add_to_calendar",
        func=add_to_calendar,
        description="Adds a task to the calendar"
    )
]

agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)

# Input for the agent
input_text = "Search for the cricket or football match today. if there is a match then displays the match details otherwise schedule a meeting"

# Plan and execute the task
result = agent(input_text)

print(result)



> Entering new AgentExecutor chain...
I need to search for the cricket or football match today and see if there is a match happening.
Action: web_search
Action Input: "cricket match today" or "football match today"
Observation: Check live score, live cricket scorecard, upcoming cricket matches and latest news and videos for today's live cricket matches on ESPN cricinfo. Cricket Matches; Football Matches; All Matches; India Matches. T20 WC 2024 ... Team India have a chance to clinch the five-match T20I series against Zimbabwe today, as they play the fourth T20I. Cricket Matches; Football Matches; All Matches; India Matches. T20 WC 2024 ... Get the Latest India tour of Zimbabwe, 2024 Updates, check out cricket news and Euro Cup 2024 at NDTV Sports. Follow our live cricket update for in-depth match coverage and exciting highlights from India vs Pakistan 12th Match in Ahmedabad on ESPNcricinfo. Follow our live cricket update for in-depth match coverage and exciting highlights from India 

## Chains & Memory

- Large Language Models are stateless.
- Each transaction is independent
- Chatbots appear to have memory by providing the full conversation as context


### LCEL (Langchain Expression Language)

In [29]:
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory

llm = ChatOpenAI(model="gpt-3.5-turbo")

memory = ConversationBufferMemory(memory_key="chat_history", output_key="response")

prompt = PromptTemplate(
    input_variables=["user_input", "history"],
    template="You are a helpful assistant. You have the following conversation history: {history}. Respond to the user's input: {user_input}"
)

chain = prompt | llm

user_inputs = [
    "Hi, I need help with my account.",
    "I forgot my password.",
    "Can you help me reset it?",
    "username: arslanhaider95"
]

for user_input in user_inputs:
    response = chain.invoke({"user_input": user_input, "history": memory.load_memory_variables({}).get("chat_history", "")})
    print("Input:", user_input)
    print("Response:", response.content + " \n")
    memory.save_context({"user_input": user_input}, {"response": []})

Input: Hi, I need help with my account.
Response: Of course! I'm here to help. What specific issue are you experiencing with your account? 

Input: I forgot my password.
Response: I can help you with that. Please provide me with your account information so I can assist you in resetting your password. 

Input: Can you help me reset it?
Response: AI: Of course! I can help you reset your password. Please provide me with your account information so we can proceed with the reset. 

Input: username: arslanhaider95
Response: I'm sorry, but I am unable to help with password resets through this platform. Please contact the customer support team for assistance with resetting your password. 



In [31]:
# Display the entire conversation history
print("\nComplete Conversation History:\n")
print(memory.load_memory_variables({})["chat_history"])


Complete Conversation History:

Human: Hi, I need help with my account.
AI: []
Human: I forgot my password.
AI: []
Human: Can you help me reset it?
AI: []
Human: username: arslanhaider95
AI: []


### Conversation Buffer Memory 

In [5]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory


llm = ChatOpenAI(temperature=0.0)

memory = ConversationBufferMemory()

conversation = ConversationChain(
    llm=llm,
    memory=memory,
    verbose=True
)

In [6]:
conversation.predict(input="Hi, My name is Arslan")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, My name is Arslan
AI:

> Finished chain.


"Hello Arslan! It's nice to meet you. How can I assist you today?"

In [7]:
conversation.predict(input="What is 1 + 1?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, My name is Arslan
AI: Hello Arslan! It's nice to meet you. How can I assist you today?
Human: What is 1 + 1?
AI:

> Finished chain.


'1 + 1 equals 2. Is there anything else you would like to know?'

In [9]:
conversation.predict(input="What is my name?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, My name is Arslan
AI: Hello Arslan! It's nice to meet you. How can I assist you today?
Human: What is 1 + 1?
AI: 1 + 1 equals 2. Is there anything else you would like to know?
Human: What is my name?
AI:

> Finished chain.


'Your name is Arslan.'

In [11]:
print(memory.buffer)

Human: Hi, My name is Arslan
AI: Hello Arslan! It's nice to meet you. How can I assist you today?
Human: What is 1 + 1?
AI: 1 + 1 equals 2. Is there anything else you would like to know?
Human: What is my name?
AI: Your name is Arslan.


In [13]:
memory.load_memory_variables({})

{'history': "Human: Hi, My name is Arslan\nAI: Hello Arslan! It's nice to meet you. How can I assist you today?\nHuman: What is 1 + 1?\nAI: 1 + 1 equals 2. Is there anything else you would like to know?\nHuman: What is my name?\nAI: Your name is Arslan."}